# 电影推荐之FM模型
[预备理论知识](https://zhuanlan.zhihu.com/p/145436595)  
[参考代码](https://github.com/morenjiujiu/FM)

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import roc_auc_score, confusion_matrix
from collections import Counter


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
path = '/home/gavin/Machine/data/ml-1m/'

## 数据分析

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv(path+'users.dat', sep='::', header=None, names=unames)
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zip         6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [4]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(path+'ratings.dat', sep='::',
                        header=None, names=rnames, engine='python')
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [6]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(path+'movies.dat', sep='::',
                       header=None, names=mnames, engine='python')
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genres    3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [8]:
# 合并数据
df = pd.merge(pd.merge(ratings, users, on='user_id'), movies, on='movie_id')
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   user_id     1000209 non-null  int64 
 1   movie_id    1000209 non-null  int64 
 2   rating      1000209 non-null  int64 
 3   timestamp   1000209 non-null  int64 
 4   gender      1000209 non-null  object
 5   age         1000209 non-null  int64 
 6   occupation  1000209 non-null  int64 
 7   zip         1000209 non-null  object
 8   title       1000209 non-null  object
 9   genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 83.9+ MB


In [10]:
# 数据量比较大，单纯为了验证FM的效果，这里挑选有20～35次打分的movie_id
# 构造2分类数据集
df = df[df['rating'] != 3]

movie_group = df.groupby('movie_id').size()
movie_group = pd.Series(movie_group).where(lambda x: x < 35).dropna()
movie_list = pd.Series(movie_group).where(
    lambda x: x > 20).dropna().index.values
len(movie_list)

322

In [11]:
df = df[df['movie_id'].isin(movie_list)]
df.shape

(8635, 10)

In [12]:
# 这里挑选有>20次打分行为的user_id
user_group = df.groupby('user_id').size()
user_list = pd.Series(user_group).where(lambda x: x > 20).dropna().index.values
len(user_list)

37

In [13]:
df = df[df['user_id'].isin(user_list)]
df.shape

(1251, 10)

In [14]:
df['user_id'].nunique(), df['movie_id'].nunique()  # 一共37个人，评分的总电影数是317部

(37, 317)

In [15]:
# 构建二分类标签
df['rating'] = df['rating'].map(
    lambda x: -1 if x > 3 else 1)  # 1,2是label=1  4,5是label=0

## 数据预处理

In [16]:
# 异常处理
df[['age']].describe([0.01, 0.05, 0.1, 0.25, 0.5, 0.75,
                      0.9, 0.99]).T  # age是被分段后的，没有异常值

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,99%,max
age,1251.0,30.801759,10.550785,1.0,1.0,18.0,18.0,25.0,25.0,35.0,50.0,50.0,50.0


In [17]:
# one-hot encoder
columns = ['user_id', 'movie_id']

for i in columns:
    get_dummy_feature = pd.get_dummies(df[i])
    df = pd.concat([df, get_dummy_feature], axis=1)
    df = df.drop(i, axis=1)

df.head()

,rating,timestamp,gender,age,occupation,zip,title,genres,195,752,850,889,1069,1150,1181,1242,1266,1448,1470,1680,1698,1733,1941,2106,2665,2909,3032,3224,3391,3476,3526,3539,3610,3829,4161,4169,4227,4277,4344,4510,4647,5074,5111,5795,5812,27,38,40,54,55,77,83,99,103,113,121,149,179,182,201,202,211,214,220,238,240,263,268,269,278,294,297,301,304,320,324,335,341,343,347,384,386,391,409,418,422,433,443,452,453,460,463,470,472,476,488,495,496,498,499,510,568,571,583,600,602,619,621,656,665,669,704,705,710,711,718,722,731,746,760,806,808,813,824,840,841,846,862,869,875,885,956,957,964,970,984,990,1044,1053,1068,1112,1113,1119,1123,1151,1169,1310,1312,1317,1322,1325,1329,1331,1413,1415,1417,1428,1439,1447,1463,1472,1473,1489,1490,1507,1508,1529,1547,1551,1563,1564,1585,1596,1602,1609,1650,1668,1669,1677,1695,1696,1755,1771,1791,1798,1809,1812,1826,1829,1844,1854,1859,1867,1869,1896,1902,1910,1922,1926,1932,2007,2031,2032,2044,2045,2049,2103,2104,2131,2179,2190,2205,2237,2284,2286,2307,2323,2341,2345,2400,2426,2449,2465,2477,2479,2481,2483,2493,2494,2523,2536,2537,2552,2566,2573,2579,2586,2593,2610,2620,2626,2630,2635,2638,2646,2653,2659,2663,2666,2681,2689,2704,2711,2744,2758,2766,2810,2812,2813,2835,2847,2849,2864,2876,2878,2884,2893,2938,2939,2977,2979,2982,2992,3002,3010,3014,3021,3073,3109,3124,3128,3132,3133,3143,3144,3158,3192,3218,3222,3223,3239,3276,3306,3311,3318,3329,3331,3338,3339,3343,3367,3389,3390,3395,3414,3434,3442,3446,3454,3496,3518,3520,3539,3540,3559,3569,3573,3586,3588,3597,3602,3605,3612,3620,3632,3659,3665,3667,3673,3674,3691,3711,3719,3731,3736,3739,3765,3780,3781,3782,3797,3805,3806,3816,3820,3838,3851,3855,3870,3884,3885,3886,3898,3900,3912,3913,3921,3922,3931,3938,3942,3951
169417,1,975254346,M,45,20,10024,Mr. Jones (1993),Drama|Romance,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
169419,1,974875561,F,25,20,75226,Mr. Jones (1993),Drama|Romance,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
169421,-1,976552318,F,25,3,17522,Mr. Jones (1993),Drama|Romance,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
df.shape

(1251, 362)

In [19]:
# label encoder
from sklearn.preprocessing import LabelEncoder
columns = ['gender', 'zip', 'genres']

for i in columns:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])

df.head()

,rating,timestamp,gender,age,occupation,zip,title,genres,195,752,850,889,1069,1150,1181,1242,1266,1448,1470,1680,1698,1733,1941,2106,2665,2909,3032,3224,3391,3476,3526,3539,3610,3829,4161,4169,4227,4277,4344,4510,4647,5074,5111,5795,5812,27,38,40,54,55,77,83,99,103,113,121,149,179,182,201,202,211,214,220,238,240,263,268,269,278,294,297,301,304,320,324,335,341,343,347,384,386,391,409,418,422,433,443,452,453,460,463,470,472,476,488,495,496,498,499,510,568,571,583,600,602,619,621,656,665,669,704,705,710,711,718,722,731,746,760,806,808,813,824,840,841,846,862,869,875,885,956,957,964,970,984,990,1044,1053,1068,1112,1113,1119,1123,1151,1169,1310,1312,1317,1322,1325,1329,1331,1413,1415,1417,1428,1439,1447,1463,1472,1473,1489,1490,1507,1508,1529,1547,1551,1563,1564,1585,1596,1602,1609,1650,1668,1669,1677,1695,1696,1755,1771,1791,1798,1809,1812,1826,1829,1844,1854,1859,1867,1869,1896,1902,1910,1922,1926,1932,2007,2031,2032,2044,2045,2049,2103,2104,2131,2179,2190,2205,2237,2284,2286,2307,2323,2341,2345,2400,2426,2449,2465,2477,2479,2481,2483,2493,2494,2523,2536,2537,2552,2566,2573,2579,2586,2593,2610,2620,2626,2630,2635,2638,2646,2653,2659,2663,2666,2681,2689,2704,2711,2744,2758,2766,2810,2812,2813,2835,2847,2849,2864,2876,2878,2884,2893,2938,2939,2977,2979,2982,2992,3002,3010,3014,3021,3073,3109,3124,3128,3132,3133,3143,3144,3158,3192,3218,3222,3223,3239,3276,3306,3311,3318,3329,3331,3338,3339,3343,3367,3389,3390,3395,3414,3434,3442,3446,3454,3496,3518,3520,3539,3540,3559,3569,3573,3586,3588,3597,3602,3605,3612,3620,3632,3659,3665,3667,3673,3674,3691,3711,3719,3731,3736,3739,3765,3780,3781,3782,3797,3805,3806,3816,3820,3838,3851,3855,3870,3884,3885,3886,3898,3900,3912,3913,3921,3922,3931,3938,3942,3951
169417,1,975254346,1,45,20,1,Mr. Jones (1993),45,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
169419,1,974875561,0,25,20,20,Mr. Jones (1993),45,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
169421,-1,976552318,0,25,3,4,Mr. Jones (1993),45,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16944

In [20]:
df = df.drop(['timestamp', 'gender', 'age', 'occupation',
              'zip', 'title', 'genres'], axis=1)
# 这些特征可以进一步挖掘。这里都不要了，只保留one-hot特征

In [21]:
# 保存数据
df.to_csv("./data/process.csv", index=False)

## FM模型

In [22]:
def sigmoid(x):

    return 1 / (1 + np.exp(-x))


def logit(y, y_hat):
    """交叉熵损失"""

    return np.log(1 + np.exp(-y * y_hat))


def df_logit(y, y_hat):

    return sigmoid(-y * y_hat) * (-y)

In [23]:
class FactorizationMachine():
    def __init__(self, k=5, learning_rate=0.01, iternum=100):

        self.w0 = None
        self.W = None
        self.V = None
        self.k = k
        self.alpha = learning_rate
        self.iternum = iternum

    def _FM(self, Xi):
        """计算FM的部分"""

        interaction = np.sum((Xi.dot(self.V)) ** 2 -
                             (Xi ** 2).dot(self.V ** 2))
        # 偏差项+一阶项＋二阶项
        y_hat = self.w0 + Xi.dot(self.W) + interaction / 2
        return y_hat[0]

    def _FM_SGD(self, X, y):
        m, n = np.shape(X)
        # 初始化参数
        self.w0 = 0
        self.W = np.random.uniform(size=(n, 1))
        # Vj是第j个特征的隐向量  Vjf是第j个特征的隐向量表示中的第f维
        self.V = np.random.uniform(size=(n, self.k))

        for it in tqdm(range(self.iternum)):
            total_loss = 0
            for i in range(m):  # 遍历训练集
                y_hat = self._FM(Xi=X[i])  # X[i]是第i个样本  X[i,j]是第i个样本的第j个特征
                total_loss += logit(y=y[i], y_hat=y_hat)  # 计算logit损失函数值
                dloss = df_logit(y=y[i], y_hat=y_hat)  # 计算logit损失函数的外层偏导
                # 更新偏差项目
                dloss_w0 = dloss * 1   # 公式中的w0求导，计算复杂度O(1)
                self.w0 = self.w0 - self.alpha * dloss_w0
                # 更新一阶项和隐向量的权重
                for j in range(n):
                    if X[i, j] != 0:
                        dloss_Wj = dloss * X[i, j]  # 公式中的wi求导，计算复杂度O(n)
                        self.W[j] = self.W[j] - self.alpha * dloss_Wj
                        for f in range(self.k):  # 公式中的vjf求导，计算复杂度O(kn)
                            dloss_Vjf = dloss * \
                                (X[i, j] * (X[i].dot(self.V[:, f])) -
                                 self.V[j, f] * X[i, j] ** 2)
                            self.V[j, f] = self.V[j, f] - \
                                self.alpha * dloss_Vjf

            if it % 20 == 0:
                print('iter={}, loss={:.4f}'.format(it, total_loss / m))

        return self

    def _FM_predict(self, X):

        predicts, threshold = [], 0.5  # sigmoid阈值设置
        for i in range(X.shape[0]):  # 遍历测试集
            y_hat = self._FM(Xi=X[i])  # FM的模型方程
            predicts.append(-1 if sigmoid(y_hat) < threshold else 1)
        return np.array(predicts)

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = np.array(X)
            y = np.array(y)

        return self._FM_SGD(X, y)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = np.array(X)

        return self._FM_predict(X)

    def predict_proba(self, X):
        pass

## 训练和预测模型

### 加载数据

In [24]:
# 划分数据集
df = pd.read_csv("./data/process.csv")
X = df.drop('rating', axis=1)
Y = df['rating']
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state=2022
                                                  )
X_train.shape, X_val.shape

((875, 354), (376, 354))

In [25]:
%%time
model = FactorizationMachine(k=10, learning_rate=0.001, iternum=150)
model.fit(X_train, Y_train)

  1%|          | 1/150 [00:00<01:40,  1.49it/s]

iter=0, loss=1.7394


 14%|█▍        | 21/150 [00:07<00:44,  2.88it/s]

iter=20, loss=0.6105


 27%|██▋       | 41/150 [00:16<00:50,  2.18it/s]

...

iter=120, loss=0.4055


 94%|█████████▍| 141/150 [01:03<00:04,  2.17it/s]

iter=140, loss=0.3890


100%|██████████| 150/150 [01:07<00:00,  2.23it/s]

CPU times: user 2min 7s, sys: 3.89 s, total: 2min 11s
Wall time: 1min 7s


### 训练集评估

In [26]:
y_pred = model.predict(X_train)
print('训练集AUC: {:.2%}'.format(roc_auc_score(Y_train.values, y_pred)))
print('混淆矩阵: \n', confusion_matrix(Y_train.values, y_pred))

训练集AUC: 85.37%
混淆矩阵: 
 [[368  65]
 [ 63 379]]


### 验证集评估

In [27]:
y_pred = model.predict(X_val)
print('验证集AUC: {:.2%}'.format(roc_auc_score(Y_val.values, y_pred)))
print('混淆矩阵: \n', confusion_matrix(Y_val.values, y_pred))

验证集AUC: 77.56%
混淆矩阵: 
 [[135  48]
 [ 36 157]]
